<a href="https://colab.research.google.com/github/ChibuzoKelechi/GoogleColab_notebooks/blob/main/sentiment_classifier_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow import keras as tfkeras
from keras import layers, losses
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
reddit_data = '/content/drive/MyDrive/Datasets/Twitter_reddit SA data/Reddit_Data.csv'
reddit

dataset = pd.read_csv(reddit_data)

comments = dataset['clean_comment'].dropna()
categories = dataset['category'].dropna()

sentences = []
labels = []

for sentence in comments:
  # sentences.append(str(dataset['clean_comment'].tolist()))
  sentences.append(str(sentence))

for category in categories:
  labels.append(category)

train_size = 28000
vocab_size = 48244
embedding_dim = 50
max_length = 100
trunc_type='post'
pad_type='post'
training_size = 20000

train_text = sentences[0:train_size]
test_text = sentences[train_size:]

train_labels = labels[0:train_size]
test_labels = labels[train_size:]

In [4]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')

tokenizer.fit_on_texts(train_text)
word_index = tokenizer.word_index

# test_sentence = ['I definitely love Machine learning', 'What is really the problem?']

train_sequence = tokenizer.texts_to_sequences(train_text)
trunc_type='post'
train_padded = pad_sequences(train_sequence, padding='post', maxlen=100, truncating=trunc_type)

test_sequence = tokenizer.texts_to_sequences(test_text)
test_padded = pad_sequences(test_sequence, padding='post', maxlen=100, truncating=trunc_type)

train_padded = np.array(train_padded)
train_labels = np.array(train_labels)
test_padded = np.array(test_padded)
test_labels = np.array(test_labels)

train_labels = train_labels.reshape(-1, 1)
test_labels = test_labels.reshape(-1, 1)

test_labels = test_labels[:len(test_padded)]

dataset_shape = dataset.shape
train_padded.shape



(28000, 100)

In [6]:
text_model = tfkeras.Sequential([
    layers.Embedding(vocab_size, embedding_dim, input_length=100),
    layers.GlobalAveragePooling1D(),
    # layers.Flatten(),
    layers.Dense(10, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

text_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# callback_func = tfkeras.Callbacks([

# ])

In [8]:
history = text_model.fit(train_padded, train_labels,
    epochs=10,
    validation_data=(test_padded, test_labels),
    verbose=1
)

Epoch 1/50
875/875 - 34s - loss: 0.5272 - accuracy: 0.3375 - val_loss: 0.4757 - val_accuracy: 0.3991 - 34s/epoch - 39ms/step
Epoch 2/50
875/875 - 29s - loss: 0.5052 - accuracy: 0.3372 - val_loss: 0.4773 - val_accuracy: 0.3991 - 29s/epoch - 33ms/step
Epoch 3/50
875/875 - 28s - loss: 0.4761 - accuracy: 0.3400 - val_loss: 0.4802 - val_accuracy: 0.3987 - 28s/epoch - 31ms/step
Epoch 4/50
875/875 - 28s - loss: 0.3411 - accuracy: 0.3726 - val_loss: 0.5563 - val_accuracy: 0.3950 - 28s/epoch - 32ms/step
Epoch 5/50
875/875 - 27s - loss: 0.0760 - accuracy: 0.4216 - val_loss: 0.7173 - val_accuracy: 0.3912 - 27s/epoch - 31ms/step
Epoch 6/50
875/875 - 28s - loss: -3.1953e-01 - accuracy: 0.4579 - val_loss: 1.0602 - val_accuracy: 0.3944 - 28s/epoch - 32ms/step
Epoch 7/50
875/875 - 28s - loss: -8.9372e-01 - accuracy: 0.4785 - val_loss: 1.5272 - val_accuracy: 0.3929 - 28s/epoch - 32ms/step
Epoch 8/50
875/875 - 28s - loss: -1.7291e+00 - accuracy: 0.4916 - val_loss: 2.3359 - val_accuracy: 0.3921 - 28s/epo